# Overview - Merge in Collapsed Executive Compensation

Based off *IRS Form 990 e-File Data (9a) -- Merge in Collapsed Executive Compensation.ipynb.ipynb*

Read in main 990 DF:
- `D:/990_and_bmf_april_2025_all_controls_351875_orgs_2598477_filings_no_duplicates_fixed_state_ntee_zip.feather`
- See `IRS 990 e-File Data -- IRS Files (9b2) -- Rationalize Geo Variables.ipynb`

Read in Schedule J Part I DF 
- *Schedule J (Part I) - parsed.pkl.gz*
- See *IRS 990 e-File Data -- IRS Files (9d) -- Schedule J Part (I) -- Verify New Variables and Combine and Binarize Columns.ipynb*

Read in DF with Schedule J Part II (highest-paid executive):
- *Schedule J Part II Variables for Highest Salary per Filing (N=743,685).pkl.gz*
- See *IRS 990 e-File Data -- IRS Files (9g) -- Schedule J Part (II) -- Generate Collapsed Filing-Level DF.ipynb*

Merge the two Schedule J files into our main dataset and save in three formats:
- `990 and BMF and Exec Comp control variables for ALL filings as of April 2025 -- 351,875 501c3 orgs -- duplicated filings dropped and fixed state and ZIP and FIPS (N=2,598,477).feather`
- `990 and BMF and Exec Comp control variables for ALL filings as of April 2025 -- 351,875 501c3 orgs -- duplicated filings dropped and fixed state and ZIP and FIPS (N=2,598,477).pkl.gz`
- `990 and BMF and Exec Comp control variables for ALL filings as of April 2025 -- 351,875 501c3 orgs -- duplicated filings dropped and fixed state and ZIP and FIPS (N=2,598,477).csv`

Note:
- Not all of the filings in the Schedule I DF merge in; the reason is that the Schedule J DFs include filings by non-501(c)(3) organizations
- Same with the Schedule II DF

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [2]:
print(pd.__version__)

2.2.2


In [3]:
from platform import python_version
print(python_version())

3.10.11


In [4]:
# http://pandas.pydata.org/pandas-docs/stable/options.html
pd.set_option('display.max_columns', None)
# http://pandas.pydata.org/pandas-docs/stable/options.html
pd.set_option('display.max_colwidth', 500)

In [5]:
#pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.display.float_format = '{:,.1f}'.format  #THIS ONE ADDS IN COMMAS FOR 1,000s

In [6]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
warnings.simplefilter(action='ignore', category=pd.errors.SettingWithCopyWarning)

#### Set working directory

In [7]:
cd "C:\\Users\\Gregory\\IRS 990 Control Variables\\"

C:\Users\Gregory\IRS 990 Control Variables


# Read in PANDAS DF

In [8]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df = pd.read_feather('D:/990_and_bmf_april_2025_all_controls_351875_orgs_2598477_filings_no_duplicates_fixed_state_ntee_zip.feather')
print('# of columns:', len(df.columns))
print('# of observations:', len(df))
df[:1]

Current date and time :  2025-06-27 14:12:17 

# of columns: 362
# of observations: 2598477
CPU times: total: 1min 10s
Wall time: 38.9 s


,EIN,F9_00_HD_TAX_YEAR,_id,OrganizationName,URL,DLN,TaxPeriod,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,fiscal_year,Name,NameControl,Phone,USAddress,ForeignAddress,InCareOfName,BusinessName,BusinessNameControlTxt,PhoneNum,InCareOfNm,ForeignPhoneNum,F9_00_HD_ADDR_CHANGE,F9_00_HD_AMENDED_RETURN,F9_00_HD_CTRY_OF_DOMICILE,F9_00_HD_EXEMPT_STATUS_4847A1,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_EXEMPT_STATUS_501C3,F9_00_HD_FINAL_RETURN,F9_00_HD_GROSS_EXEMPT_NUM,F9_00_HD_GROSS_RCPT,F9_00_HD_GROUP_RETURN,F9_00_HD_INCLUDES_SUBORD_ORGS,F9_00_HD_INITIAL_RETURN,F9_00_HD_PRIN_OFF_NAME,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_SPECIAL_CONDITION_DESC,F9_00_HD_STATE_OF_DOMICILE,F9_00_HD_TAX_PER_BEGIN,F9_00_HD_TAX_PER_END,F9_00_HD_TIME_STAMP,F9_00_HD_TYPE_ORG_ASSOCIATION,F9_00_HD_TYPE_ORG_CORP,F9_00_HD_TYPE_ORG_OTHER,F9_00_HD_TYPE_ORG_OTHER_DESC,F9_00_HD_TYPE_ORG_TRUST,F9_00_HD_WEBSITE,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TERMINATION_CONTRACTION,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_ORGANIZATIONAL_MISSION,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PGMSVC_SIGNIF_CHG,F9_03_PC_PGMSVC_SIGNIF_NEW,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_DESC,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_03_PZ_MISSION_DESCRIPTION,F9_03_PZ_SCHEDULE_O_PART3,F9_04_PC_ACTVITIES_VIA_PARTNER,F9_04_PC_CONTROLLED_ENTITY,F9_04_PC_DISREGARDED_ENTITY,F9_04_PC_EXCESS_BENEFIT_TRANS,F9_04_PC_FR_EVENT_INC_GT_15K,F9_04_PC_GAMING_INC_GT_15K,F9_04_PC_LOBBYING_ACTIVITIES,F9_04_PC_POLITICAL_ACTIVITIES,F9_04_PC_PRIOR_EXCESS_BEN_TRAN,F9_04_PC_PROF_FR_EXP_GT_15K,F9_04_PC_RELATED_ENTITY,F9_04_PC_TRANS_TO_CNTRLD_ENT,F9_04_PC_TRANS_WITH_CNTRLD_ENT,F9_05_EXP_SCHED_O_X,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_05_PC_UNRELATED_BUS_INCOME,F9_06_EXP_SCHED_O_X,F9_06_PC_990_PROVIDED_GOV_BODY,F9_06_PC_ANNUAL_DISC_COVRD_PERS,F9_06_PC_CEO_COMPENSTN_PROCESS,F9_06_PC_CHANGES_ORGANIZING_DOCS,F9_06_PC_CONFLICT_OF_INTEREST,F9_06_PC_DECISIONS_SUBJ_APPROVAL,F9_06_PC_DELEGATION_MGT_DUTIES,F9_06_PC_DELEGATION_OF_MGT,F9_06_PC_DOCUMENT_RET_POLICY,F9_06_PC_ELECTION_BOARD_MEMBERS,F9_06_PC_FAMILY_OR_BUSINESS_REL,F9_06_PC_FORM_AVAIL_OWN_WEBSITE,F9_06_PC_FORM_UPON_REQUEST,F9_06_PC_JOINT_VENTURE_INVESTMNT,F9_06_PC_JOINT_VENTURE_POLICY,F9_06_PC_LOCAL_CHAPTERS,F9_06_PC_MATERIAL_DIVERSION,F9_06_PC_MEMBERS_OR_STOCKHOLDERS,F9_06_PC_MINUTES_COMMITTEES,F9_06_PC_MINUTES_GOVERNING_BODY,F9_06_PC_MONITORING_OF_COI_POLICY,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_06_PC_OFFICER_MAILING_ADDRESS,F9_06_PC_OTHER_COMPENSTN_PROCESS,F9_06_PC_OTHER_WEBSITE,F9_06_PC_OWN_WEBSITE,F9_06_PC_POLICIES_GOVERN_CHAPTER,F9_06_PC_STATES_WHERE_RET_FILED,F9_

#### Sidebar -- look for county-level donations

In [9]:
[c for c in df.columns if 'code' in c.lower()]

['F9_03_PC_PROG_SVC_ACC_1_CODE',
 'F9_03_PC_PROG_SVC_ACC_2_CODE',
 'F9_03_PC_PROG_SVC_ACC_3_CODE',
 'BMF_GEOCODER_SCORE',
 'BMF_GEOCODER_MATCH',
 'BMF_BMF_SUBSECTION_CODE',
 'BMF_BMF_STATUS_CODE',
 'BMF_BMF_PF_FILING_REQ_CODE',
 'BMF_BMF_ORGANIZATION_CODE',
 'BMF_BMF_INCOME_CODE',
 'BMF_BMF_FOUNDATION_CODE',
 'BMF_BMF_FILING_REQ_CODE',
 'BMF_BMF_DEDUCTIBILITY_CODE',
 'BMF_BMF_CLASSIFICATION_CODE',
 'BMF_BMF_ASSET_CODE',
 'BMF_BMF_AFFILIATION_CODE',
 'COUNTY_CODE']

In [10]:
[c for c in df.columns if 'year' in c.lower()]

['F9_00_HD_TAX_YEAR',
 'fiscal_year',
 'F9_00_HD_YEAR_FORMED',
 'BMF_ORG_FISCAL_YEAR',
 'BMF_ORG_RULING_YEAR',
 'BMF_ORG_YEAR_FIRST',
 'BMF_ORG_YEAR_LAST',
 'BMF_ORG_YEAR_COUNT',
 'filing_year_had_duplicate']

In [18]:
df['F9_00_HD_TAX_YEAR'].value_counts()

F9_00_HD_TAX_YEAR
2021    246349
2020    238428
2019    214705
2018    200443
2017    191330
2016    182883
2022    178528
2015    175083
2014    163692
2013    149129
2012    134483
2011    119339
2010     91917
2009     28080
2023       123
Name: count, dtype: int64

In [11]:
df['direct_donations'] = df['F9_08_PC_TOTAL_CONTRIBUTIONS']-df['F9_08_PC_FEDERATED_CAMPAIGNS'] -df['F9_08_PC_GOVERNMENT_GRANTS']

In [12]:
df['direct_donations'].info()

<class 'pandas.core.series.Series'>
Index: 2314512 entries, 0 to 2314511
Series name: direct_donations
Non-Null Count    Dtype  
--------------    -----  
2314512 non-null  float64
dtypes: float64(1)
memory usage: 35.3 MB


In [13]:
print(len(df['COUNTY_CODE'].tolist()))
print(len(set(df['COUNTY_CODE'].tolist())))
print(len(df[df['COUNTY_CODE'].notnull()]))
fips = list(set(df[df['COUNTY_CODE'].notnull()]['COUNTY_CODE'].tolist()))
print(len(fips))
fips[:10]

2314512
6525
2311162
3175


['51199',
 '36115',
 '51790',
 '18153',
 '22025',
 '24035',
 '38099',
 '46047',
 '27087',
 '53011']

In [20]:
dfg = df[df['F9_00_HD_TAX_YEAR']==2021][['BMF_FIPS', 'direct_donations']].groupby('BMF_FIPS').sum()
dfg.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3130 entries, 00000 to 90724
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   direct_donations  3130 non-null   float64
dtypes: float64(1)
memory usage: 48.9+ KB


In [21]:
print(len(dfg))

3130


In [22]:
dfg.sample(10)

,direct_donations
BMF_FIPS,
17005,"7,241,499.0"
20093,"165,200.0"
27063,"125,727.0"
18149,"438,634.0"
12035,"6,453,637.0"
20099,"5,790,583.0"
21121,"2,191,525.0"
40001,"815,283.0"
17081,"6,923,392.0"


In [23]:
print(len(dfg[dfg['direct_donations']==0]))
print(len(dfg[dfg['direct_donations']>0]))

37
3093


# Read in Schedule J Part I

In [11]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df1 = pd.read_pickle('Schedule J (Part I) - parsed.pkl.gz', compression='gzip')
print('# of columns:', len(df1.columns))
print('# of observations:', len(df1))
df1[:1]

Current date and time :  2025-06-27 14:14:04 

# of columns: 27
# of observations: 687646
CPU times: total: 422 ms
Wall time: 1.51 s


,URL,SJ_01_PC_BOARD_APPROVAL,SJ_01_PC_CLUB_FEES,SJ_01_PC_COMPANION_TRAVEL,SJ_01_PC_COMPENSATION_COMMITTEE,SJ_01_PC_COMPENSATION_SURVEY,SJ_01_PC_CONSULTANT,SJ_01_PC_CONTINGENT_NET_OWN,SJ_01_PC_CONTINGENT_NET_RELATED,SJ_01_PC_CONTINGENT_REV_OWN,SJ_01_PC_CONTINGENT_REV_RELATED,SJ_01_PC_CONTRACT,SJ_01_PC_CONTRACT_EXCEPTION,SJ_01_PC_DISCRETIONARY_ACCOUNT,SJ_01_PC_EQUITY_BASED_COMP,SJ_01_PC_FIRST_CLASS_TRAVEL,SJ_01_PC_HOME_OFFICE_SUBSIDY,SJ_01_PC_HOUSING_ALLOWANCE,SJ_01_PC_INDEMNIFICATION,SJ_01_PC_NON_FIXED_PAYMENTS,SJ_01_PC_OTHER_ORGS_990,SJ_01_PC_PERSONAL_SERVICES,SJ_01_PC_REBUTTABLE_PRESUMPTION,SJ_01_PC_SEVERANCE,SJ_01_PC_SUBSTANTIATION_REQUIRED,SJ_01_PC_SUPPLEMENTAL_RETIREMENT,SJ_01_PC_WRITTEN_POLICY
0,https://s3.amazonaws.com/irs-form-990/201113139349301311_public.xml,1.0,NaN,NaN,1.0,1.0,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,NaN,1.0,NaN


In [12]:
df1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 687646 entries, 0 to 687645
Data columns (total 27 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   URL                               687646 non-null  object 
 1   SJ_01_PC_BOARD_APPROVAL           398308 non-null  float64
 2   SJ_01_PC_CLUB_FEES                39872 non-null   float64
 3   SJ_01_PC_COMPANION_TRAVEL         28539 non-null   float64
 4   SJ_01_PC_COMPENSATION_COMMITTEE   198519 non-null  float64
 5   SJ_01_PC_COMPENSATION_SURVEY      254666 non-null  float64
 6   SJ_01_PC_CONSULTANT               98304 non-null   float64
 7   SJ_01_PC_CONTINGENT_NET_OWN       549972 non-null  float64
 8   SJ_01_PC_CONTINGENT_NET_RELATED   549903 non-null  float64
 9   SJ_01_PC_CONTINGENT_REV_OWN       550010 non-null  float64
 10  SJ_01_PC_CONTINGENT_REV_RELATED   549811 non-null  float64
 11  SJ_01_PC_CONTRACT                 154351 non-null  f

# Read in DF with Schedule J Part II for Highest-paid Executive

In [13]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df2 = pd.read_pickle('Schedule J Part II Variables for Highest Salary per Filing (N=743,685).pkl.gz', compression='gzip')
print('# of columns:', len(df2.columns))
print('# of observations:', len(df2))
df2[:1]

Current date and time :  2025-06-27 14:14:45 

# of columns: 19
# of observations: 743685
CPU times: total: 1.23 s
Wall time: 1.35 s


,URL,max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP,max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP_L1,max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP_L2,max_comp_SJ_02_PC_TITLE,max_comp_SJ_02_PC_COMP_BASE,max_comp_SJ_02_PC_COMP_BASE_RELATED,max_comp_SJ_02_PC_COMP_BONUS,max_comp_SJ_02_PC_COMP_BONUS_RELATED,max_comp_SJ_02_PC_COMP_OTHER,max_comp_SJ_02_PC_COMP_OTHER_RELATED,max_comp_SJ_02_PC_COMP_DEFERRED,max_comp_SJ_02_PC_COMP_DEFERRED_RELATED,max_comp_SJ_02_PC_NONTAXED_BENF,max_comp_SJ_02_PC_NONTAXED_BENF_RELATED,max_comp_SJ_02_PC_COMP_TOTAL,max_comp_SJ_02_PC_COMP_TOTAL_RELATED,max_comp_SJ_02_PC_COMP_DEF_PRIOR,max_comp_SJ_02_PC_COMP_DEF_PRIOR_RELATED
2756613,https://s3.amazonaws.com/irs-form-990/201610749349300436_public.xml,KATHY CAMPBELL,NaN,NaN,PRESIDENT/CEO,"17,511,315,603.0",NaN,"19,080.0",NaN,NaN,NaN,"26,011.0",NaN,NaN,NaN,"17,511,360,694.0",NaN,NaN,NaN


# Merge

In [14]:
set(df.columns.tolist()).intersection(set(df1.columns.tolist()))

{'URL'}

In [15]:
set(df.columns.tolist()).intersection(set(df2.columns.tolist()))

{'URL'}

In [16]:
set(df1.columns.tolist()).intersection(set(df2.columns.tolist()))

{'URL'}

In [17]:
%%time
pd.merge(df, df1, left_on='URL', right_on='URL', how='left', indicator=True)['_merge'].value_counts()

CPU times: total: 28.1 s
Wall time: 29.9 s


_merge
left_only     2070768
both           527709
right_only          0
Name: count, dtype: int64

##### The reason not all of the Schedule J filings merged in is that it includes filings by non-501(c)(3) organizations

In [18]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
print(len(df))
print(len(df.columns))
print(len(df1))
print(len(df1.columns))
print(len(pd.merge(df, df1, left_on='URL', right_on='URL', how='left', indicator=True)))
dfm = pd.merge(df, df1, left_on='URL', right_on='URL', how='left', indicator=True)
print(len(dfm))
print(len(dfm.columns))
print(dfm['_merge'].value_counts())
dfm = dfm.drop('_merge', axis=1)
dfm[:2]

Current date and time :  2025-06-27 14:16:53 

2598477
362
687646
27
2598477
2598477
389
_merge
left_only     2070768
both           527709
right_only          0
Name: count, dtype: int64
CPU times: total: 48.7 s
Wall time: 50.2 s


,EIN,F9_00_HD_TAX_YEAR,_id,OrganizationName,URL,DLN,TaxPeriod,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,fiscal_year,Name,NameControl,Phone,USAddress,ForeignAddress,InCareOfName,BusinessName,BusinessNameControlTxt,PhoneNum,InCareOfNm,ForeignPhoneNum,F9_00_HD_ADDR_CHANGE,F9_00_HD_AMENDED_RETURN,F9_00_HD_CTRY_OF_DOMICILE,F9_00_HD_EXEMPT_STATUS_4847A1,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_EXEMPT_STATUS_501C3,F9_00_HD_FINAL_RETURN,F9_00_HD_GROSS_EXEMPT_NUM,F9_00_HD_GROSS_RCPT,F9_00_HD_GROUP_RETURN,F9_00_HD_INCLUDES_SUBORD_ORGS,F9_00_HD_INITIAL_RETURN,F9_00_HD_PRIN_OFF_NAME,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_SPECIAL_CONDITION_DESC,F9_00_HD_STATE_OF_DOMICILE,F9_00_HD_TAX_PER_BEGIN,F9_00_HD_TAX_PER_END,F9_00_HD_TIME_STAMP,F9_00_HD_TYPE_ORG_ASSOCIATION,F9_00_HD_TYPE_ORG_CORP,F9_00_HD_TYPE_ORG_OTHER,F9_00_HD_TYPE_ORG_OTHER_DESC,F9_00_HD_TYPE_ORG_TRUST,F9_00_HD_WEBSITE,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TERMINATION_CONTRACTION,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_ORGANIZATIONAL_MISSION,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PGMSVC_SIGNIF_CHG,F9_03_PC_PGMSVC_SIGNIF_NEW,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_DESC,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_03_PZ_MISSION_DESCRIPTION,F9_03_PZ_SCHEDULE_O_PART3,F9_04_PC_ACTVITIES_VIA_PARTNER,F9_04_PC_CONTROLLED_ENTITY,F9_04_PC_DISREGARDED_ENTITY,F9_04_PC_EXCESS_BENEFIT_TRANS,F9_04_PC_FR_EVENT_INC_GT_15K,F9_04_PC_GAMING_INC_GT_15K,F9_04_PC_LOBBYING_ACTIVITIES,F9_04_PC_POLITICAL_ACTIVITIES,F9_04_PC_PRIOR_EXCESS_BEN_TRAN,F9_04_PC_PROF_FR_EXP_GT_15K,F9_04_PC_RELATED_ENTITY,F9_04_PC_TRANS_TO_CNTRLD_ENT,F9_04_PC_TRANS_WITH_CNTRLD_ENT,F9_05_EXP_SCHED_O_X,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_05_PC_UNRELATED_BUS_INCOME,F9_06_EXP_SCHED_O_X,F9_06_PC_990_PROVIDED_GOV_BODY,F9_06_PC_ANNUAL_DISC_COVRD_PERS,F9_06_PC_CEO_COMPENSTN_PROCESS,F9_06_PC_CHANGES_ORGANIZING_DOCS,F9_06_PC_CONFLICT_OF_INTEREST,F9_06_PC_DECISIONS_SUBJ_APPROVAL,F9_06_PC_DELEGATION_MGT_DUTIES,F9_06_PC_DELEGATION_OF_MGT,F9_06_PC_DOCUMENT_RET_POLICY,F9_06_PC_ELECTION_BOARD_MEMBERS,F9_06_PC_FAMILY_OR_BUSINESS_REL,F9_06_PC_FORM_AVAIL_OWN_WEBSITE,F9_06_PC_FORM_UPON_REQUEST,F9_06_PC_JOINT_VENTURE_INVESTMNT,F9_06_PC_JOINT_VENTURE_POLICY,F9_06_PC_LOCAL_CHAPTERS,F9_06_PC_MATERIAL_DIVERSION,F9_06_PC_MEMBERS_OR_STOCKHOLDERS,F9_06_PC_MINUTES_COMMITTEES,F9_06_PC_MINUTES_GOVERNING_BODY,F9_06_PC_MONITORING_OF_COI_POLICY,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_06_PC_OFFICER_MAILING_ADDRESS,F9_06_PC_OTHER_COMPENSTN_PROCESS,F9_06_PC_OTHER_WEBSITE,F9_06_PC_OWN_WEBSITE,F9_06_PC_POLICIES_GOVERN_CHAPTER,F9_06_PC_STATES_WHERE_RET_FILED,F9_

##### Merge in *df2*

In [19]:
%%time
pd.merge(dfm, df2, left_on='URL', right_on='URL', how='left', indicator=True)['_merge'].value_counts()

CPU times: total: 27.3 s
Wall time: 27.9 s


_merge
left_only     2026806
both           571671
right_only          0
Name: count, dtype: int64

In [20]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
print(len(dfm))
print(len(dfm.columns))
print(len(df2))
print(len(df2.columns))
print(len(pd.merge(dfm, df2, left_on='URL', right_on='URL', how='left', indicator=True)))
dfm = pd.merge(dfm, df2, left_on='URL', right_on='URL', how='left', indicator=True)
print(len(dfm))
print(len(dfm.columns))
print(dfm['_merge'].value_counts())
dfm = dfm.drop('_merge', axis=1)
dfm[:2]

Current date and time :  2025-06-27 14:19:43 

2598477
388
743685
19
2598477
2598477
407
_merge
left_only     2026806
both           571671
right_only          0
Name: count, dtype: int64
CPU times: total: 52 s
Wall time: 54 s


,EIN,F9_00_HD_TAX_YEAR,_id,OrganizationName,URL,DLN,TaxPeriod,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,fiscal_year,Name,NameControl,Phone,USAddress,ForeignAddress,InCareOfName,BusinessName,BusinessNameControlTxt,PhoneNum,InCareOfNm,ForeignPhoneNum,F9_00_HD_ADDR_CHANGE,F9_00_HD_AMENDED_RETURN,F9_00_HD_CTRY_OF_DOMICILE,F9_00_HD_EXEMPT_STATUS_4847A1,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_EXEMPT_STATUS_501C3,F9_00_HD_FINAL_RETURN,F9_00_HD_GROSS_EXEMPT_NUM,F9_00_HD_GROSS_RCPT,F9_00_HD_GROUP_RETURN,F9_00_HD_INCLUDES_SUBORD_ORGS,F9_00_HD_INITIAL_RETURN,F9_00_HD_PRIN_OFF_NAME,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_SPECIAL_CONDITION_DESC,F9_00_HD_STATE_OF_DOMICILE,F9_00_HD_TAX_PER_BEGIN,F9_00_HD_TAX_PER_END,F9_00_HD_TIME_STAMP,F9_00_HD_TYPE_ORG_ASSOCIATION,F9_00_HD_TYPE_ORG_CORP,F9_00_HD_TYPE_ORG_OTHER,F9_00_HD_TYPE_ORG_OTHER_DESC,F9_00_HD_TYPE_ORG_TRUST,F9_00_HD_WEBSITE,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TERMINATION_CONTRACTION,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_ORGANIZATIONAL_MISSION,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PGMSVC_SIGNIF_CHG,F9_03_PC_PGMSVC_SIGNIF_NEW,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_DESC,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_03_PZ_MISSION_DESCRIPTION,F9_03_PZ_SCHEDULE_O_PART3,F9_04_PC_ACTVITIES_VIA_PARTNER,F9_04_PC_CONTROLLED_ENTITY,F9_04_PC_DISREGARDED_ENTITY,F9_04_PC_EXCESS_BENEFIT_TRANS,F9_04_PC_FR_EVENT_INC_GT_15K,F9_04_PC_GAMING_INC_GT_15K,F9_04_PC_LOBBYING_ACTIVITIES,F9_04_PC_POLITICAL_ACTIVITIES,F9_04_PC_PRIOR_EXCESS_BEN_TRAN,F9_04_PC_PROF_FR_EXP_GT_15K,F9_04_PC_RELATED_ENTITY,F9_04_PC_TRANS_TO_CNTRLD_ENT,F9_04_PC_TRANS_WITH_CNTRLD_ENT,F9_05_EXP_SCHED_O_X,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_05_PC_UNRELATED_BUS_INCOME,F9_06_EXP_SCHED_O_X,F9_06_PC_990_PROVIDED_GOV_BODY,F9_06_PC_ANNUAL_DISC_COVRD_PERS,F9_06_PC_CEO_COMPENSTN_PROCESS,F9_06_PC_CHANGES_ORGANIZING_DOCS,F9_06_PC_CONFLICT_OF_INTEREST,F9_06_PC_DECISIONS_SUBJ_APPROVAL,F9_06_PC_DELEGATION_MGT_DUTIES,F9_06_PC_DELEGATION_OF_MGT,F9_06_PC_DOCUMENT_RET_POLICY,F9_06_PC_ELECTION_BOARD_MEMBERS,F9_06_PC_FAMILY_OR_BUSINESS_REL,F9_06_PC_FORM_AVAIL_OWN_WEBSITE,F9_06_PC_FORM_UPON_REQUEST,F9_06_PC_JOINT_VENTURE_INVESTMNT,F9_06_PC_JOINT_VENTURE_POLICY,F9_06_PC_LOCAL_CHAPTERS,F9_06_PC_MATERIAL_DIVERSION,F9_06_PC_MEMBERS_OR_STOCKHOLDERS,F9_06_PC_MINUTES_COMMITTEES,F9_06_PC_MINUTES_GOVERNING_BODY,F9_06_PC_MONITORING_OF_COI_POLICY,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_06_PC_OFFICER_MAILING_ADDRESS,F9_06_PC_OTHER_COMPENSTN_PROCESS,F9_06_PC_OTHER_WEBSITE,F9_06_PC_OWN_WEBSITE,F9_06_PC_POLICIES_GOVERN_CHAPTER,F9_06_PC_STATES_WHERE_RET_FILED,F9_

In [21]:
dfm.columns[-45:]

Index(['ZIP5', 'SJ_01_PC_BOARD_APPROVAL', 'SJ_01_PC_CLUB_FEES',
       'SJ_01_PC_COMPANION_TRAVEL', 'SJ_01_PC_COMPENSATION_COMMITTEE',
       'SJ_01_PC_COMPENSATION_SURVEY', 'SJ_01_PC_CONSULTANT',
       'SJ_01_PC_CONTINGENT_NET_OWN', 'SJ_01_PC_CONTINGENT_NET_RELATED',
       'SJ_01_PC_CONTINGENT_REV_OWN', 'SJ_01_PC_CONTINGENT_REV_RELATED',
       'SJ_01_PC_CONTRACT', 'SJ_01_PC_CONTRACT_EXCEPTION',
       'SJ_01_PC_DISCRETIONARY_ACCOUNT', 'SJ_01_PC_EQUITY_BASED_COMP',
       'SJ_01_PC_FIRST_CLASS_TRAVEL', 'SJ_01_PC_HOME_OFFICE_SUBSIDY',
       'SJ_01_PC_HOUSING_ALLOWANCE', 'SJ_01_PC_INDEMNIFICATION',
       'SJ_01_PC_NON_FIXED_PAYMENTS', 'SJ_01_PC_OTHER_ORGS_990',
       'SJ_01_PC_PERSONAL_SERVICES', 'SJ_01_PC_REBUTTABLE_PRESUMPTION',
       'SJ_01_PC_SEVERANCE', 'SJ_01_PC_SUBSTANTIATION_REQUIRED',
       'SJ_01_PC_SUPPLEMENTAL_RETIREMENT', 'SJ_01_PC_WRITTEN_POLICY',
       'max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP',
       'max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP_L1',
       'max_comp_SJ

In [22]:
dfm[['SJ_01_PC_BOARD_APPROVAL', 'SJ_01_PC_CLUB_FEES',
       'SJ_01_PC_COMPANION_TRAVEL', 'SJ_01_PC_COMPENSATION_COMMITTEE',
       'SJ_01_PC_COMPENSATION_SURVEY', 'SJ_01_PC_CONSULTANT',
       'SJ_01_PC_CONTINGENT_NET_OWN', 'SJ_01_PC_CONTINGENT_NET_RELATED',
       'SJ_01_PC_CONTINGENT_REV_OWN', 'SJ_01_PC_CONTINGENT_REV_RELATED',
       'SJ_01_PC_CONTRACT', 'SJ_01_PC_CONTRACT_EXCEPTION',
       'SJ_01_PC_DISCRETIONARY_ACCOUNT', 'SJ_01_PC_EQUITY_BASED_COMP',
       'SJ_01_PC_FIRST_CLASS_TRAVEL', 'SJ_01_PC_HOME_OFFICE_SUBSIDY',
       'SJ_01_PC_HOUSING_ALLOWANCE', 'SJ_01_PC_INDEMNIFICATION',
       'SJ_01_PC_NON_FIXED_PAYMENTS', 'SJ_01_PC_OTHER_ORGS_990',
       'SJ_01_PC_PERSONAL_SERVICES', 'SJ_01_PC_REBUTTABLE_PRESUMPTION',
       'SJ_01_PC_SEVERANCE', 'SJ_01_PC_SUBSTANTIATION_REQUIRED',
       'SJ_01_PC_SUPPLEMENTAL_RETIREMENT', 'SJ_01_PC_WRITTEN_POLICY',
       'max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP',
       'max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP_L1',
       'max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP_L2', 'max_comp_SJ_02_PC_TITLE',
       'max_comp_SJ_02_PC_COMP_BASE', 'max_comp_SJ_02_PC_COMP_BASE_RELATED',
       'max_comp_SJ_02_PC_COMP_BONUS', 'max_comp_SJ_02_PC_COMP_BONUS_RELATED',
       'max_comp_SJ_02_PC_COMP_OTHER', 'max_comp_SJ_02_PC_COMP_OTHER_RELATED',
       'max_comp_SJ_02_PC_COMP_DEFERRED',
       'max_comp_SJ_02_PC_COMP_DEFERRED_RELATED',
       'max_comp_SJ_02_PC_NONTAXED_BENF',
       'max_comp_SJ_02_PC_NONTAXED_BENF_RELATED',
       'max_comp_SJ_02_PC_COMP_TOTAL', 'max_comp_SJ_02_PC_COMP_TOTAL_RELATED',
       'max_comp_SJ_02_PC_COMP_DEF_PRIOR',
       'max_comp_SJ_02_PC_COMP_DEF_PRIOR_RELATED']].info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2598477 entries, 0 to 2598476
Data columns (total 44 columns):
 #   Column                                     Non-Null Count   Dtype  
---  ------                                     --------------   -----  
 0   SJ_01_PC_BOARD_APPROVAL                    302693 non-null  float64
 1   SJ_01_PC_CLUB_FEES                         29372 non-null   float64
 2   SJ_01_PC_COMPANION_TRAVEL                  16045 non-null   float64
 3   SJ_01_PC_COMPENSATION_COMMITTEE            158140 non-null  float64
 4   SJ_01_PC_COMPENSATION_SURVEY               204689 non-null  float64
 5   SJ_01_PC_CONSULTANT                        80654 non-null   float64
 6   SJ_01_PC_CONTINGENT_NET_OWN                526763 non-null  float64
 7   SJ_01_PC_CONTINGENT_NET_RELATED            526675 non-null  float64
 8   SJ_01_PC_CONTINGENT_REV_OWN                526814 non-null  float64
 9   SJ_01_PC_CONTINGENT_REV_RELATED            526627 non-null  float64
 10  SJ_01_

In [23]:
dfm[['SJ_01_PC_BOARD_APPROVAL', 'SJ_01_PC_CLUB_FEES',
       'SJ_01_PC_COMPANION_TRAVEL', 'SJ_01_PC_COMPENSATION_COMMITTEE',
       'SJ_01_PC_COMPENSATION_SURVEY', 'SJ_01_PC_CONSULTANT',
       'SJ_01_PC_CONTINGENT_NET_OWN', 'SJ_01_PC_CONTINGENT_NET_RELATED',
       'SJ_01_PC_CONTINGENT_REV_OWN', 'SJ_01_PC_CONTINGENT_REV_RELATED',
       'SJ_01_PC_CONTRACT', 'SJ_01_PC_CONTRACT_EXCEPTION',
       'SJ_01_PC_DISCRETIONARY_ACCOUNT', 'SJ_01_PC_EQUITY_BASED_COMP',
       'SJ_01_PC_FIRST_CLASS_TRAVEL', 'SJ_01_PC_HOME_OFFICE_SUBSIDY',
       'SJ_01_PC_HOUSING_ALLOWANCE', 'SJ_01_PC_INDEMNIFICATION',
       'SJ_01_PC_NON_FIXED_PAYMENTS', 'SJ_01_PC_OTHER_ORGS_990',
       'SJ_01_PC_PERSONAL_SERVICES', 'SJ_01_PC_REBUTTABLE_PRESUMPTION',
       'SJ_01_PC_SEVERANCE', 'SJ_01_PC_SUBSTANTIATION_REQUIRED',
       'SJ_01_PC_SUPPLEMENTAL_RETIREMENT', 'SJ_01_PC_WRITTEN_POLICY',
       'max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP',
       'max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP_L1',
       'max_comp_SJ_02_PC_NAME_OFF_TRST_KEYEMP_L2', 'max_comp_SJ_02_PC_TITLE',
       'max_comp_SJ_02_PC_COMP_BASE', 'max_comp_SJ_02_PC_COMP_BASE_RELATED',
       'max_comp_SJ_02_PC_COMP_BONUS', 'max_comp_SJ_02_PC_COMP_BONUS_RELATED',
       'max_comp_SJ_02_PC_COMP_OTHER', 'max_comp_SJ_02_PC_COMP_OTHER_RELATED',
       'max_comp_SJ_02_PC_COMP_DEFERRED',
       'max_comp_SJ_02_PC_COMP_DEFERRED_RELATED',
       'max_comp_SJ_02_PC_NONTAXED_BENF',
       'max_comp_SJ_02_PC_NONTAXED_BENF_RELATED',
       'max_comp_SJ_02_PC_COMP_TOTAL', 'max_comp_SJ_02_PC_COMP_TOTAL_RELATED',
       'max_comp_SJ_02_PC_COMP_DEF_PRIOR',
       'max_comp_SJ_02_PC_COMP_DEF_PRIOR_RELATED']].describe().T

,count,mean,std,min,25%,50%,75%,max
SJ_01_PC_BOARD_APPROVAL,"302,693.0",1.0,0.0,1.0,1.0,1.0,1.0,1.0
SJ_01_PC_CLUB_FEES,"29,372.0",1.0,0.0,1.0,1.0,1.0,1.0,1.0
SJ_01_PC_COMPANION_TRAVEL,"16,045.0",1.0,0.0,1.0,1.0,1.0,1.0,1.0
SJ_01_PC_COMPENSATION_COMMITTEE,"158,140.0",1.0,0.0,1.0,1.0,1.0,1.0,1.0
SJ_01_PC_COMPENSATION_SURVEY,"204,689.0",1.0,0.0,1.0,1.0,1.0,1.0,1.0
SJ_01_PC_CONSULTANT,"80,654.0",1.0,0.0,1.0,1.0,1.0,1.0,1.0
SJ_01_PC_CONTINGENT_NET_OWN,"526,763.0",0.0,0.1,0.0,0.0,0.0,0.0,1.0
SJ_01_PC_CONTINGENT_NET_RELATED,"526,675.0",0.0,0.1,0.0,0.0,0.0,0.0,1.0
SJ_01_PC_CONTINGENT_REV_OWN,"526,814.0",0.0,0.1,0.0,0.0,0.0,0.0,1.0
SJ_01_PC_CONTINGENT_REV_RELATED,"526,627.0",0.0,0.1,0.0,0.0,0.0,0.0,1.0


#### Save DF

In [24]:
print(len(dfm))
print(len(set(dfm['EIN'].tolist())))

2598477
351875


In [25]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
print('# of columns:', len(df.columns))
print('# of observations:', len(df))
dfm.to_feather('990 and BMF and Exec Comp control variables for ALL filings as of April 2025 -- 351,875 501c3 orgs -- duplicated filings dropped and fixed state and ZIP and FIPS (N=2,598,477).feather')

Current date and time :  2025-06-27 14:24:09 

# of columns: 362
# of observations: 2598477
CPU times: total: 44.4 s
Wall time: 55.2 s


In [27]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
print('# of columns:', len(df.columns))
print('# of observations:', len(df))
dfm.to_pickle('990 and BMF and Exec Comp control variables for ALL filings as of April 2025 -- 351,875 501c3 orgs -- duplicated filings dropped and fixed state and ZIP and FIPS (N=2,598,477).pkl.gz',
            compression = 'gzip')

Current date and time :  2025-06-27 14:27:49 

# of columns: 362
# of observations: 2598477
CPU times: total: 37min 52s
Wall time: 39min 53s


In [26]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
print('# of columns:', len(df.columns))
print('# of observations:', len(df))
dfm.to_pickle('990 and BMF and Exec Comp control variables for ALL filings as of April 2025 -- 351,875 501c3 orgs -- duplicated filings dropped and fixed state and ZIP and FIPS (N=2,598,477).csv')

Current date and time :  2025-06-27 14:25:23 

# of columns: 362
# of observations: 2598477
CPU times: total: 22.6 s
Wall time: 2min 25s
